In [1]:
import torch
from fast_bert.data_cls import BertDataBunch
from fast_bert.learner_cls import BertLearner
from fast_bert.data_lm import BertLMDataBunch
from fast_bert.learner_lm import BertLMLearner
from fast_bert.metrics import fbeta, roc_auc
from fast_bert.prediction import BertClassificationPredictor
from pathlib import Path
import pandas as pd
import logging
import unicodedata

logger = logging.getLogger()

I0308 17:49:55.527483  7628 file_utils.py:41] PyTorch version 1.4.0+cpu available.


In [2]:
import torch
camembert = torch.hub.load('pytorch/fairseq', 'camembert.v0')

Using cache found in C:\Users\luism/.cache\torch\hub\pytorch_fairseq_master
I0308 17:50:07.078038  7628 file_utils.py:274] http://dl.fbaipublicfiles.com/fairseq/models/camembert.v0.tar.gz not found in cache, downloading to C:\Users\luism\AppData\Local\Temp\tmpwtfsjhz1
100%|██████████████████████████████████████████████████████████████████| 1012251671/1012251671 [01:39<00:00, 10191120.16B/s]
I0308 17:51:46.896458  7628 file_utils.py:287] copying C:\Users\luism\AppData\Local\Temp\tmpwtfsjhz1 to cache at C:\Users\luism\.cache\torch\pytorch_fairseq\df8ea5d155cb66cfb63da3aea2b4e72963253193e2355dc97c94e3442753b1ee.6c562bd0a54059b12e9c1a2085a6bea85d93196761f431a0df3a95c80f7a0dd5
I0308 17:51:49.925221  7628 file_utils.py:291] creating metadata file for C:\Users\luism\.cache\torch\pytorch_fairseq\df8ea5d155cb66cfb63da3aea2b4e72963253193e2355dc97c94e3442753b1ee.6c562bd0a54059b12e9c1a2085a6bea85d93196761f431a0df3a95c80f7a0dd5
I0308 17:51:49.927221  7628 file_utils.py:298] removing temp file C:\Us

ValueError: invalid literal for int() with base 10: '#fairseq:overwrite\n'

In [ ]:
from utils import get_raw_data

In [ ]:
train_data, test_data, y_train, G, train_hosts, test_hosts = get_raw_data(encoding="utf-8")

In [ ]:
train_data = [unicodedata.normalize('NFKD', text).encode("utf-8").decode('utf-8','ignore').encode("latin-1", "ignore").decode("utf-8", "ignore").replace(",","") for text in train_data] #.encode('ascii', 'ignore').decode("utf-8")
test_data = [unicodedata.normalize('NFKD', text).encode("utf-8").decode('utf-8','ignore').encode("latin-1", "ignore").decode("utf-8", "ignore").replace(",","") for text in test_data] # .encode('ascii', 'ignore').decode("utf-8")

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
idx_train, idx_valid = train_test_split(np.arange(len(train_data)), stratify=y_train, test_size=.15)
train = [train_data[idx] for idx in idx_train]
valid = [train_data[idx] for idx in idx_valid]

In [ ]:
import keras
max_words = 10000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, 
                                              char_level=False)
tokenize.fit_on_texts(train_data) # fit tokenizer to our training text data
x_train = tokenize.texts_to_matrix(train)
x_valid = tokenize.texts_to_matrix(valid)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(y_train)
print(labels.shape)
onehot_enc = OneHotEncoder()
labels = onehot_enc.fit_transform(labels.reshape(-1, 1))
train_labels = labels[idx_train]
valid_labels = labels[idx_valid]

In [ ]:
labels = labels.todense()

# Data Formatting

In [ ]:
# Pre-Process and dump
import pandas as pd
with open("./labels/labels.csv", "w") as file:
    file.write("\n".join(label_encoder.classes_))

In [ ]:
pd.DataFrame({'caption':train_data}).to_csv("./data/raw_data.csv", encoding="latin-1")

In [ ]:
to_dump = {"review":train_data}
for i, label in enumerate(label_encoder.classes_):
    to_dump[label] = np.array(labels[:,i]).ravel()

In [ ]:
to_dump.keys()

In [ ]:
to_dump['sports']

In [ ]:
to_dump = pd.DataFrame(to_dump)
to_dump.index.name = 'id'

In [ ]:
to_dump.to_csv("./data/labeled_data.csv", encoding="latin-1")

In [ ]:
DATA_PATH = Path('./data/')
LOG_PATH = Path('./logs/')
MODEL_PATH = Path('./model/')
LABEL_PATH = Path('./labels/')

In [ ]:
help(unicodedata.normalize)

In [ ]:
unicodedata.normalize('NFKD', train_data[0])[:200]

# ML

In [ ]:
import spacy
device = 'cpu'
all_texts = train_data + test_data
databunch_lm = BertLMDataBunch.from_raw_corpus(
                    data_dir=DATA_PATH,
                    text_list=all_texts,
                    tokenizer='camembert-base',
                    batch_size_per_gpu=16,
                    max_seq_length=512,
                    multi_gpu=False,
                    model_type='camembert-base',
                    logger=logger)

lm_learner = BertLMLearner.from_pretrained_model(
                            dataBunch=databunch_lm,
                            pretrained_path='camembert-base',
                            output_dir=MODEL_PATH,
                            metrics=[],
                            device=device_cuda,
                            logger=logger,
                            multi_gpu=False,
                            logging_steps=50,
                            fp16_opt_level="O2")

lm_learner.fit(epochs=30,
            lr=1e-4,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

In [ ]:
lm_learner.validate()

In [ ]:
learner.save_model()

In [ ]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='camembert-base',
                          train_file='train_set.csv',
                          val_file='val_set.csv',
                          label_file='labels.txt',
                          text_col='review',
                          label_col=['cadre/atmosphère','probleme technique',"temps d'attente",'accueil/relation commerciale'],
                          batch_size_per_gpu=16,
                          max_seq_length=512,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='camembert-base')